In [ ]:
from thetaray.api.evaluation import evaluate_step
from thetaray.api.evaluation import write_evaluated_activities, load_evaluated_activities
from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
import datetime

ROWS = 20000
SCORE_THRESHOLD = 0.8

In [ ]:
spark_conf={
    "spark.executor.memory": "2g",
    "spark.executor.cores": "1",
    "spark.executor.instances": "2",
    "spark.sql.broadcastTimeout": "600"
}
context = init_context(execution_date=datetime.datetime(1970,1,1), spark_conf=spark_conf)
spark = context.get_spark_session()

In [ ]:
detect_df = dataset_functions.read(context, 'wrangling')
# detect_df = detect_df.drop('response', 'tr_timestamp')
detect_df = detect_df.drop('tr_timestamp')

In [ ]:
enriched_df, _, _ = evaluate_step(context, 'tr_analysis', 'tr_evaluation', detect_df)

In [ ]:
anomalies_df = enriched_df.filter(enriched_df.tr_evaluation_score > 0.45)

In [ ]:
# len(anomalies_df.columns)
# anomalies_df.show(10)

In [ ]:
write_evaluated_activities(context, anomalies_df, 'tr_analysis')

In [ ]:
load_evaluated_activities(context, 'tr_analysis').show(10)

In [ ]:
spark.stop()